In [1]:
# !pip show spacy-transformers transformers
# !pip install transformers==4.36.0
# !pip install gensim

In [2]:
from gensim import corpora,models,similarities
from sklearn.datasets import fetch_20newsgroups as getData
from sklearn.model_selection import train_test_split
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')
import re
import numpy as np

In [3]:
corpus = getData(subset = 'train',
                    remove=('header','footer','quotes'),
                    categories=['sci.electronics',
                                'rec.motorcycles',
                               'comp.graphics',
                               'talk.religion.misc',
                               'talk.politics.guns'])

# corpus = getData(subset='train', 
#                              remove=('headers', 'footers', 'quotes'),
#                             categories=['alt.atheism','comp.graphics',
#                                        'soc.religion.christian'])
    
X = corpus.data
y = corpus.target
y_names = corpus.target_names

In [4]:
print(X[0],len(X),y_names[y[0]])

Organization: Penn State University
From: <LRR105@psuvm.psu.edu>
Subject: Re: Tools Tools Tools
 <1993Apr1.162709.16643@osf.org> <1993Apr2.235809.3241@kronos.arc.nasa.gov>
 <1993Apr5.165548.21479@research.nj.nec.com>
Lines: 1

WHAT IS THE FLANK DRIVE EVERYONES TALKING ABOUT?
 2696 rec.motorcycles


In [5]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.1,shuffle=True)

In [6]:
stoplist = set('for a of the and to in'.split(' '))
texts = [[word for word in re.split('\W+',doc.lower()) if word not in stoplist] for doc in Xtrain]

In [7]:
print(len(texts))

2426


In [8]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

cutOffValue = 10
processed_corpus = [[token for token in text if frequency[token] > cutOffValue] 
                    for text in texts]

In [9]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<5023 unique tokens: ['', '13', 'am', 'an', 'any']...>


In [10]:
print(dictionary.token2id)

{'': 0, '13': 1, 'am': 2, 'an': 3, 'any': 4, 'be': 5, 'changes': 6, 'companies': 7, 'could': 8, 'distribution': 9, 'edu': 10, 'electrical': 11, 'email': 12, 'engineering': 13, 'engr': 14, 'from': 15, 'have': 16, 'hello': 17, 'highly': 18, 'host': 19, 'i': 20, 'if': 21, 'info': 22, 'it': 23, 'its': 24, 'john': 25, 'jp': 26, 'latech': 27, 'lines': 28, 'looking': 29, 'louisiana': 30, 'make': 31, 'material': 32, 'may': 33, 'me': 34, 'names': 35, 'nntp': 36, 'on': 37, 'or': 38, 'organization': 39, 'other': 40, 'p': 41, 'posting': 42, 'property': 43, 'reply': 44, 'rubber': 45, 'some': 46, 'stuff': 47, 'subject': 48, 'tech': 49, 'thanx': 50, 'that': 51, 'university': 52, 'usa': 53, 'when': 54, 'would': 55, 'you': 56, '02': 57, '04': 58, '1': 59, '10': 60, '11': 61, '12': 62, '15': 63, '17': 64, '19': 65, '1993': 66, '2': 67, '20': 68, '26': 69, '37': 70, '5': 71, '93': 72, '___': 73, 'all': 74, 'are': 75, 'around': 76, 'at': 77, 'attached': 78, 'big': 79, 'blue': 80, 'buck': 81, 'buffalo': 82

In [11]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]  #bow = bag of corpus

In [12]:
print(bow_corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 3), (11, 2), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 3), (28, 1), (29, 1), (30, 2), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 3), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 2)]


In [13]:
model = models.TfidfModel(bow_corpus)
#model = models.LsiModel(bow_corpus)
#model = models.LdaModel(bow_corpus)

In [14]:
index = similarities.SparseMatrixSimilarity(model[bow_corpus],num_features=len(dictionary))

In [20]:
t = np.random.randint(len(Xtest))
query_document = re.split('\W+',Xtest[t].lower())
query_bow = dictionary.doc2bow(query_document)
sims = index[model[query_bow]]
docNumber = sorted(enumerate(sims), key=lambda x: x[1], reverse=True)[0][0]
print('Predicted:',y_names[ytrain[docNumber]])
print('Ground Truth:',y_names[ytest[t]])

Predicted: comp.graphics
Ground Truth: comp.graphics
